In [1]:
LAB_USER_NAME = 'DEMO'

**Important**: To initialise this notebook, edit the cell above to set `LAB_USER_NAME` to your name, then click **Run->Run All Cells** in the top menu bar.

In [2]:
import panel as pn
pn.extension(raw_css=['''progress {margin: 0;}''']) # raw_css setting is a workaround for panel issue 4112
import sys
import os
import numpy as np

# add inline dashboard libraries to path so they can be imported later
sys.path.append('../../../dashboards-inline')

LAB_DIR = os.path.join('/home/data/', LAB_USER_NAME)
os.makedirs(LAB_DIR, exist_ok=True)
print('User data directory:', LAB_DIR)

User data directory: /home/data/DEMO


### Lab 2 structure:

#### After this lab, the student should have learned (optimistically):
- How the precession frequency at different locations is affected during a magnetic field gradient
<details><summary>Explanation</summary>
    The precession frequency will have an offset proportional to the coordinate along the gradient axis
</details>
- How the precession phase at different locations is affected after a gradient pulse
<details><summary>Explanation</summary>
    The precession phase will be shifted by an amount proportional to the area under the gradient pulse and the coordinate along the gradient axis
</details>
- How frequency encoding allows position to be measured
<details><summary>Explanation</summary>
    By taking the fourier transform of the signal acquired with a gradient active, the coordinate along gradient axis can be calculated from the frequency offset and gradient amplitude
</details>
- How phase encoding allows position to be measured
<details><summary>Explanation</summary>
    Applying a gradient pulse before acquisition allows measuring a single spatial frequency.
    Doing this multiple times with different gradient areas allows measuring a range of spatial frequencies, which can be used to reconstruct the image with a fourier transform.
</details>
- How the k-space spectrum relates to the image
<details><summary>Explanation</summary>
    K-space is the fourier transform of cartesian coordinate space. Coordinates in k-space are spatial frequencies, and the value at a coordinate tells you how well the image matches a sinewave with that wavelength (inverse of spatial frequency) and direction.
</details>
- How the k-space coordinate of a discrete signal sample is determined from the gradient pulse sequence
<details><summary>Explanation</summary>
    Each k-space coordinate of a sample is proportional to the area under the gradient applied along that coordinate axis between the excitation (RF 90 pulse) and the acquisition of that sample.
</details>
- What (in k-space, and in the pulse sequence) determines the field of view/zoom of the image
<details><summary>Explanation</summary>
    Field of View is determined by the spacing between samples in k-space, which is determined by the difference in area under the gradient pulses of two successive samples.
    A larger spacing results in a smaller field of view.
    For frequency encoding the k-space spacing is the gradient amplitude multiplied by the dwell time, and for phase encoding it is the gradient step size multiplied by the phase gradient duration.
</details>
- What (in k-space, and in the pulse sequence) determines the spatial resolution of the image
<details><summary>Explanation</summary>
    Spatial resolution is determined by the extent of the k-space spectrum. 
    Sampling a larger region of k-space results in a smaller (in meters, i.e. better) spatial resolution. 
    For frequency encoding the size of k-space is the gradient amplitude multiplied by the total acquisition time, and for phase encoding it is the area under the maximum gradient pulse.
</details>

#### Diagrams:
1. SE with constant shim
2. SE with Z frequency encode
3. SE with XY vector freq encode
4. SE with X frequency and Y phase encode gradients
5. ~~SE with RF soft pulse and constant shim gradient~~
6. ~~Full slice 2D SE~~

#### Samples:
1. Uniform sample (shim sample)
2. Single thin layer sample
3. 1D Z phantom with multiple alternating layers of plastic and water (e.g. 5 plastic, 4 water, height: 20mm plenty of plastic padding above and below to exclude surrounding water)
4. XY projection phantom with two vertical water cylinders in plastic
5. ~~Slice phantom?~~

#### Experiments:
1. Full acquisition spin echo with adjustable x,y,z shims.
2. 1D SE with Z frequency encode. Allow altering the gradient strength and observing the resulting signal and spectrum.
3. 1D SE with X/Y oblique frequency encode. Allow altering gradient magnitude and angle and observing the resulting signal and spectrum.
4. 2D XY SE with X frequency encode, Y phase encode. Allow adjusting X and Y gradient strengths.
5. ~~Full acquisition spin echo with adjustable width soft pulse and adjustable x,y,z shims.~~
6. ~~2D SE with slicing~~

#### Steps:
1. See how X, Y, Z shims alter the spin echo signal and spectrum, with uniform sample and 1D Z phantom
2. Run 1D SE experiment
3. Run a 2D XY SE with X frequency encode, Y phase encode.
    - Allow adjusting X and Y gradient strengths and number of samples/steps. Display Y gradient step size.
    - Display k-space and the image, updating after every acquisition so they can see how the image looks as k-space is built up.
    - Fix the kx,ky,x,y axes so they can see the size of the k-space sampling area and the image FOV changing
4. 


TODO: explain X/Y/Z shims/gradients, how they are produced and what effect they have on signal.

#### Task

Maybe make them use shim sample and see the effect of individually changing X/Y/Z first? Then move to thin layer sample.

1. Insert the thin layer sample
2. Reset shims and start experiment with "Run Loop"
3. Zoom in on the echo signal (from 6us to 14us)
4. Move the sample up and down. Q: What happens to the signal when it is moved too far from the centre?
5. Return the sample to roughly the centre (where the signal returns to full amplitude)
6. Increase the Z shim to its limit (1.0, or -1.0). Q: How does the signal shape change when the shim is offset?
7. Move the sample up and down a small amount, so that the magnitude is not significantly affected. Q: What happens to the signal frequency?
8. Click abort to stop the experiment before moving on

In [3]:
# Experiment 1: Full acquisition SE with constant shim
# load global shims file
import yaml
from matipo import GLOBALS_DIR
SHIM_ORDER = ['shim_x', 'shim_y', 'shim_z']
SHIM_FILE = os.path.join(GLOBALS_DIR, 'shims.yaml')

with open(SHIM_FILE, 'r') as f:
    SHIM_INIT = yaml.safe_load(f)

# create shim inputs, using saved user shims as initial values
shim_inputs = {}
for shim_key in SHIM_ORDER:
    shim_name = shim_key.split('_')[-1].upper()
    shim_inputs[shim_key] = pn.widgets.FloatInput(name=shim_name+' shim', start=-1, end=1, step=0.01, value=round(SHIM_INIT[shim_key], 2), width=80)

# button to reset shim inputs to the saved values
reset_btn = pn.widgets.Button(name='Reset', button_type='primary', align='end', width=100)

def reset_shim_inputs(e):
    for shim_key in SHIM_ORDER:
        shim_inputs[shim_key].value = round(SHIM_INIT[shim_key], 2)

reset_btn.on_click(reset_shim_inputs)
    
from full_acq_SE import FullAcqSEApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    t_echo=0.01,
    n_scans=1,
    n_samples=1500,
    t_dw=10e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.2
)

# add shim inputs
override_pars.update(shim_inputs)

# create dashboard app
exp1_app = FullAcqSEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=True,
    enable_run_loop=True
)

# display layout
pn.Column(
    # echo_time,
    pn.Row(*([shim_inputs[shim_key] for shim_key in SHIM_ORDER]+[reset_btn])), # take shim inputs in order and concatenate reset button
    exp1_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] Row
        [0] FloatInput(end=1, name='X shim', start=-1, step=0.01, value=-0.59, width=80)
        [1] FloatInput(end=1, name='Y shim', start=-1, step=0.01, value=-0.44, width=80)
        [2] FloatInput(end=1, name='Z shim', start=-1, step=0.01, value=0.69, width=80)
        [3] Button(align='end', button_type='primary', name='Reset', width=100)
    [1] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=31, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

TODO: frequency encoding explanation

TODO: Short reference explanations of FOV/SNR/spatial resolution (not pixel resolution)

#### Task
1. With the thin layer sample still inserted, start the experiment with "Run Loop"
2. Increase the gradient to 0.25.
3. Move the sample up and down again, and see how the spectrum changes. Q: Does this match your observations in the previous experiment?
4. Inspect the 1D Z phantom and, based on your observations with the thin layer sample, draw a rough plot of the spectrum you would expect to see when it is inserted.
5. Insert the 1D Z phantom (centred with the depth gauge). Q: Does the spectrum look as you expected?
6. Adjust the gradient strength through its possible range and note how the spectrum "image" changes. Q: Comparing the image with 0.25 gradient and 0.5 gradient: Which has a larger field of view (FOV)? Which has better signal-to-noise ratio (SNR)? Which has better spatial resolution?
7. Click abort to stop the experiment before moving on

In [13]:
# Experiment 2: SE with adjustable Z read gradient showing signal and spectrum of echo
# read gradient input
# TODO: calibrate and use mT/m
read_z_grad = pn.widgets.FloatInput(name='Read Z Gradient', start=-1, end=1, step=0.01, value=0, width=200)

# TODO: oversample and use decimation before taking the FFT for flat filter profile
from SE import SEApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    g_read=lambda: (0, 0, read_z_grad.value),
    t_echo=0.012,
    n_scans=1,
    n_samples=1000,
    t_dw=10e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.2
)

# create dashboard app
exp2_app = SEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=True,
    enable_run_loop=True
)

# display layout
pn.Column(
    read_z_grad,
    exp2_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] FloatInput(end=1, name='Read Z Gradient', start=-1, step=0.01, width=200)
    [1] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=31, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

#### Task
1. Insert the mystery sample at the correct depth and start the experiment below with "Run Loop"
2. In this experiment you can define a read gradient vector by setting the magnitude and direction (counter-clockwise rotation from $+x$ axis). Increase the magnitude to get a 1D image and try both physically rotating the sample and rotating the read gradient direction.
3. Set the gradient magnitude appropriately so that the 1D image is not cropped with any gradient direction.
4. Based on 1D views at different angles, draw a 2D top-down diagram of where you think the water is located in the mystery sample. Hint: draw a square in your notebook, and for each edge draw the 1D image you get from orienting the gradient perpendicular to that edge. Based on this you can draw inside the square where the water is located.
5. Click abort to stop the experiment before moving on

In [21]:
# Experiment 3: 1D SE with adjustable X/Y vector read gradient
# read gradient input
# TODO: calibrate and use mT/m
exp3_read_grad_mag = pn.widgets.FloatInput(name='Read Gradient Magnitude', start=0, end=1, step=0.01, value=0, width=200)
exp3_read_grad_dir = pn.widgets.FloatInput(name='Read Gradient Direction (degrees)', start=0, end=360, step=10, value=0, width=200)
exp3_read_x_grad = pn.widgets.FloatInput(name='Read Gradient X Component', start=-1, end=1, step=0.01, value=0, width=200, disabled=True)
exp3_read_y_grad = pn.widgets.FloatInput(name='Read Gradient Y Component', start=-1, end=1, step=0.01, value=0, width=200, disabled=True)

def exp3_read_grad_cb(*events):
    exp3_read_x_grad.value = exp3_read_grad_mag.value*np.cos((np.pi/180)*exp3_read_grad_dir.value)
    exp3_read_y_grad.value = exp3_read_grad_mag.value*np.sin((np.pi/180)*exp3_read_grad_dir.value)

exp3_read_grad_mag.param.watch(exp3_read_grad_cb, ['value'], onlychanged=True)
exp3_read_grad_dir.param.watch(exp3_read_grad_cb, ['value'], onlychanged=True)

# TODO: oversample and use decimation before taking the FFT for flat filter profile
from SE import SEApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    g_read=lambda: (exp3_read_grad_mag.value*np.cos((np.pi/180)*exp3_read_grad_dir.value), exp3_read_grad_mag.value*np.sin((np.pi/180)*exp3_read_grad_dir.value), 0),
    t_echo=0.012,
    n_scans=1,
    n_samples=1000,
    t_dw=10e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.2
)

# create dashboard app
exp3_app = SEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=False,
    enable_run_loop=True
)

# display layout
pn.Column(
    pn.Row(exp3_read_grad_mag, exp3_read_grad_dir, exp3_read_x_grad, exp3_read_y_grad),
    exp3_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] Row
        [0] FloatInput(end=1, name='Read Gradient Magnitude', start=0, step=0.01, width=200)
        [1] FloatInput(end=360, name='Read Gradient D..., start=0, step=10, width=200)
        [2] FloatInput(disabled=True, end=1, name='Read Gradient X..., start=-1, step=0.01, width=200)
        [3] FloatInput(disabled=True, end=1, name='Read Gradient Y..., start=-1, step=0.01, width=200)
    [1] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=31, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

TODO: explain k-space

diagrams:
- read encoding SE pulse sequence and k-space trajectory for a read gradient vector rotated 30 degrees ccw from the x axis
- phase/read encoding SE pulse sequence and k-space trajectory

#### Task
- 

In [23]:
# Experiment 4: 2D SE with adjustable X read gradient and Y phase gradient showing kspace and image
read_x_grad = pn.widgets.FloatInput(name='Read X Gradient', start=-1, end=1, step=0.01, value=0, width=200)
samples = pn.widgets.IntInput(name='Read Samples', start=1, end=200, step=1, value=32, width=200)
dwell_time = pn.widgets.IntInput(name='Dwell Time (us)', start=2, end=80, step=2, value=20, width=200)
phase_y_grad = pn.widgets.FloatInput(name='Phase Y Gradient', start=-1, end=1, step=0.01, value=0, width=200)
phase_steps = pn.widgets.IntInput(name='Phase Steps', start=1, end=200, step=1, value=32, width=200)
phase_y_width = pn.widgets.IntInput(name='Duration (us)', start=10, end=1000, step=10, value=200, width=200)

def get_g_phase():
    g_phase_max = phase_y_grad.value
    # linearly spaced gradient values
    g_phase = np.linspace(g_phase_max, -g_phase_max, phase_steps.value, endpoint=False)
    # sort by magnitude for divergent ordering
    g_phase = g_phase[np.argsort(np.abs(g_phase))]
    # multiply by Y direction vector
    g_phase = np.outer(g_phase, np.array([0, 1, 0]))
    return g_phase

import importlib
import RARE2D
importlib.reload(RARE2D)

# TODO: oversample and use decimation before taking the FFT for flat filter profile
# TODO: fix phase order so it is consistent
# TODO: change k-space colormap
from RARE2D import RARE2DApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    g_read=lambda: (read_x_grad.value, 0, 0),
    g_phase_1=get_g_phase,
    n_ETL=1,
    t_echo=0.01,
    n_scans=1,
    n_samples=samples,
    t_dw=lambda: 1e-6*dwell_time.value,
    t_read=lambda: 1e-6*dwell_time.value*samples.value,
    t_phase=lambda: 1e-6*phase_y_width.value,
    t_end=1,
)

# create dashboard app
exp4_app = RARE2DApp(
    override_pars=override_pars
)

# display layout
pn.Column(
    pn.Row(read_x_grad, samples, dwell_time),
    pn.Row(phase_y_grad, phase_steps, phase_y_width),
    exp4_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] Row
        [0] FloatInput(end=1, name='Read X Gradient', start=-1, step=0.01, width=200)
        [1] IntInput(end=200, name='Read Samples', start=1, value=32, width=200)
        [2] IntInput(end=80, name='Dwell Time (us)', start=2, step=2, value=20, width=200)
    [1] Row
        [0] FloatInput(end=1, name='Phase Y Gradient', start=-1, step=0.01, width=200)
        [1] IntInput(end=200, name='Phase Steps', start=1, value=32, width=200)
        [2] IntInput(end=1000, name='Duration (us)', start=10, step=10, value=200, width=200)
    [2] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [2] Progress(height=31, sizing_mode='stretch_width', value=0)
            [3] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

In [7]:
g_phase_max = 1
g_phase = np.linspace(g_phase_max, -g_phase_max, phase_steps.value, endpoint=False)
g_phase = np.outer(g_phase, np.array([0, 0, 1]))
g_phase = g_phase[np.argsort(np.sum(np.abs(g_phase), axis=1), axis=0)]

In [8]:
np.argsort(np.sum(g_phase, axis=1), axis=0)

array([29, 28, 25, 23, 21, 19, 17, 16, 14, 11,  9,  7,  6,  3,  2,  0,  1,
        4,  5,  8, 10, 12, 13, 15, 18, 20, 22, 24, 26, 27, 30, 31],
      dtype=int32)

In [24]:
from bokeh.models.mappers import LinearColorMapper, LogColorMapper
exp4_app.plot1.im.glyph.color_mapper = LinearColorMapper(palette="Viridis256", low=0)

In [26]:
pn.io.push_notebook(exp4_app.plot_row)

In [ ]:
exp3_app.plot1.im.glyph.color_mapper = 
exp3_app.plot1.im.glyph.color_mapper.high = None

In [ ]:
help(exp3_app.plot1.im.glyph.color_mapper)

In [ ]:
# Experiment ?: Full acquisition SE with sinc RF pulses and adjustable X/Y/Z shims

In [ ]:
# Experiment ?: 2D SE with slicing